In [0]:
%sql
create volume dev.bronze.landing
comment 'This is managed bronze landing volume'

In [0]:
%python
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/01")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/02")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/03")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/04")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/05")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/06")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/07")
dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/autoloader_input/2010/12/08")

In [0]:
# Location for checkpoint

dbutils.fs.mkdirs("/Volumes/dev/bronze/landing/checkpoint/autoloader")